與父子關係的excel進行整合

In [1]:
import pandas as pd
import os
import openpyxl
from openpyxl.styles import Alignment, Font, PatternFill

#3.1開始
def merge_and_format_excels(first_excel_path, second_excel_path, third_excel_path, output_folder, file_name):
    # 讀取第一個 Excel 文件
    df1 = pd.read_excel(first_excel_path)

    # 讀取第二個和第三個 Excel 文件
    df2 = pd.read_excel(second_excel_path)
    df3 = pd.read_excel(third_excel_path)
#3.1結束
#---------------------------------------------------------------------------------------------------------------------
#3.2開始
    # 根據你要的搜尋的欄位填入要搜尋的欄位內容為何
    df1_part_number_col = '件號\nPart Number'
    df2_part_number_col = '件號\nPart Number'
    df3_part_number_col = '件號\nPart Number'

    # 創建一個空的 DataFrame 用於存儲結果
    result_df = df1.copy()

    # 遍歷第一個 DataFrame 中的每一行，並在第二和第三個 DataFrame 中搜尋相應的資料
    for index, row in df1.iterrows():
        part_number = row[df1_part_number_col]

        # 在第二個 DataFrame 中搜索
        matching_row_2 = df2[df2[df2_part_number_col] == part_number]

        # 在第三個 DataFrame 中搜索
        matching_row_3 = df3[df3[df3_part_number_col] == part_number]

        if not matching_row_2.empty:
            result_df.at[index, '名稱\nNOMENCLA TURE'] = matching_row_2['名稱\nNOMENCLA TURE'].values[0]
            result_df.at[index, '材質\nMaterial'] = matching_row_2['材質\nMaterial'].values[0]
            result_df.at[index, '規範\nSpec.'] = matching_row_2['規範\nSpec.'].values[0]
        elif not matching_row_3.empty:
            result_df.at[index, '名稱\nNOMENCLA TURE'] = matching_row_3['名稱\nNOMENCLA TURE'].values[0]
            result_df.at[index, '材質\nMaterial'] = matching_row_3['材質\nMaterial'].values[0]
            result_df.at[index, '規範\nSpec.'] = matching_row_3['規範\nSpec.'].values[0]

    # 儲存合併後的結果到一個新的 Excel 文件
    temp_file_path = os.path.join(output_folder, f"{file_name}_temp.xlsx")
    result_df.to_excel(temp_file_path, index=False)

    # 讀取合併後的 Excel 文件並應用格式設定
    workbook = openpyxl.load_workbook(temp_file_path)
    sheet = workbook.active

    sheet.cell(row=1,column=1).value = ' '
    # 將 B 欄填入 NO
    sheet.cell(row=1, column=2).value = 'NO'

    # 將 C~K 欄填入 0~8
    for col in range(3, 12):
        sheet.cell(row=1, column=col).value = col - 3

    # 將 L 欄的標題設置為 '件號\nPart Number'
    sheet.cell(row=1, column=12).value = '件號\nPart Number'

    # 將 M 欄的標題設置為 '名稱\nNOMENCLA TURE'
    sheet.cell(row=1, column=13).value = '名稱\nNOMENCLA TURE'

    # 將 N 欄的標題設置為 '類別\nCode'
    sheet.cell(row=1, column=14).value = '類別\nCode'

    # 將 O 欄的標題設置為 '零件代碼\nShape'
    sheet.cell(row=1, column=15).value = '零件代碼\nShape'

    # 將 P 欄的標題設置為 '材質\nMaterial'
    sheet.cell(row=1, column=16).value = '材質\nMaterial'

    # 將 Q 欄的標題設置為 '規範\nSpec.'
    sheet.cell(row=1, column=17).value = '規範\nSpec.'
#3.2結束
#------------------------------------------------------------------------
#3.3開始
    # 設置對齊和自動換行
    target_row = 1
    last_col = sheet.max_column
    alignment = Alignment(horizontal='center', wrapText=True)

    for col in range(1, last_col + 1):
        cell = sheet.cell(row=target_row, column=col)
        cell.alignment = alignment

    # 設置顏色與字形
    for col_index in range(1, last_col + 1):
        cell = sheet.cell(row=1, column=col_index)
        cell.fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
        if isinstance(cell.value, str) and any('\u4e00' <= char <= '\u9fff' for char in cell.value):
            font = Font(name='Malgun Gothic', color='000000', bold=False)  # 設定要的中文字體
        else:
            font = Font(name='Calibri', color='000000', bold=False)  # 設定要的英文字體
        cell.font = font

    # 設定欄寬
    columns_to_adjust = ['L', 'M', 'P', 'Q']
    default_column_width = 30
    columns_to_adjust_1 = ['N', 'O']
    default_column_width_1 = 10

    for col_letter in columns_to_adjust:
        sheet.column_dimensions[col_letter].width = default_column_width
    for col_letter in columns_to_adjust_1:
        sheet.column_dimensions[col_letter].width = default_column_width_1

    # 儲存最終的結果
    final_file_path = os.path.join(output_folder, f"{file_name}_output.xlsx")
    workbook.save(final_file_path)
    os.remove(temp_file_path)  # 刪除臨時文件

    print(f"資料合併完成，結果已儲存到新的 Excel 文件中：{final_file_path}")

# 設定文件路徑和文件名
first_excel_path = r'父子階層MBOM-前半部.xlsx'
second_excel_path = r'-X_PL\final_output\combine.xlsx'
third_excel_path = r'_PL\final_output\combine.xlsx'
output_folder = r'final_mbom'
file_name = r'MBOM'

# 執行合併和格式設定
merge_and_format_excels(first_excel_path, second_excel_path, third_excel_path, output_folder, file_name)
#3.3結束

資料合併完成，結果已儲存到新的 Excel 文件中：final_mbom\MBOM_output.xlsx
